In [1]:
import torch
import sys
import os
import pandas as pd
import numpy as np
import shutil
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from math import ceil
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.optim import AdamW
import torch.optim as optim
from sklearn.metrics import precision_score, recall_score, f1_score
from tqdm import tqdm
from IPython.display import clear_output
import os
import cv2

from albumentations import (
    Compose, RandomResizedCrop, HorizontalFlip, VerticalFlip,
    ShiftScaleRotate, RandomBrightnessContrast, HueSaturationValue,
    RGBShift, ChannelShuffle, MotionBlur, MedianBlur, GaussianBlur,
    GaussNoise, GridDistortion, ElasticTransform, CoarseDropout, Lambda
)

In [1]:
df = pd.read_csv('train_dataset.csv')
print(df['benign_malignant'].value_counts())  # Убедимся, что замена прошла успешно

NameError: name 'pd' is not defined

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Используемое устройство: {device}")

Используемое устройство: cuda


In [3]:
print(sys.executable)  # Путь должен указывать на ваш venv

c:\Users\mdima\Desktop\derm\classification\venv\Scripts\python.exe


In [ ]:
# минимальный размер для условного ресайза
min_crop = 700

def conditional_resize(image: np.ndarray, **kwargs) -> np.ndarray:
    """Если min(width, height) < min_crop — ресайзим так, чтобы min(side) == min_crop."""
    h, w = image.shape[:2]
    if min(h, w) < min_crop:
        scale = min_crop / min(h, w)
        new_h, new_w = int(h * scale), int(w * scale)
        image = cv2.resize(image, (new_w, new_h),
                           interpolation=cv2.INTER_LINEAR)
    return image

def get_offline_augmentations(image_size: int = 256, scale: tuple[float, float] = (0.08, 1.0), ratio: tuple[float, float] = (0.75, 1.33)):
    """
    Пайплайн для оффлайновой аугментации (возвращает numpy.ndarray).
    Обновлено для albumentations >= 1.3.0
    """
    return Compose([
        # Первый этап - условный ресайз
        Lambda(image=conditional_resize, p=1.0),
        
        # Основные аугментации
        RandomResizedCrop(
            size=(image_size, image_size),
            scale=scale,
            ratio=ratio,
            interpolation=cv2.INTER_LINEAR,
            always_apply=True  # Этот параметр теперь обязательный
        ),
        HorizontalFlip(p=0.5),
        VerticalFlip(p=0.2),
        ShiftScaleRotate(
            shift_limit=0.1,
            scale_limit=0.2,
            rotate_limit=30,
            interpolation=cv2.INTER_LINEAR,
            p=0.7
        ),
        GridDistortion(
            num_steps=5,
            distort_limit=0.3,
            interpolation=cv2.INTER_LINEAR,
            p=0.3
        ),
        ElasticTransform(
            alpha=1,
            sigma=50,
            alpha_affine=50,
            interpolation=cv2.INTER_LINEAR,
            p=0.3
        ),
        RandomBrightnessContrast(
            brightness_limit=0.3,
            contrast_limit=0.3,
            p=0.7
        ),
        HueSaturationValue(
            hue_shift_limit=20,
            sat_shift_limit=30,
            val_shift_limit=20,
            p=0.5
        ),
        RGBShift(
            r_shift_limit=15,
            g_shift_limit=15,
            b_shift_limit=15,
            p=0.3
        ),
        ChannelShuffle(p=0.1),
        MotionBlur(blur_limit=7, p=0.2),
        MedianBlur(blur_limit=5, p=0.2),
        GaussianBlur(blur_limit=3, p=0.2),
        GaussNoise(var_limit=(10.0, 50.0), p=0.3),
        CoarseDropout(
            max_holes=8,
            max_height=int(image_size * 0.1),
            max_width=int(image_size * 0.1),
            fill_value=0,
            p=0.5
        ),
    ])

def split_dataset(
    img_dir: str,
    csv_path: str,
    val_dir: str,
    train_csv: str,
    val_csv: str,
    val_frac: float = 0.2,
    random_state: int = 42
) -> pd.DataFrame:
    """
    Стратифицированный сплит:
    - Читает csv (img_id,target_feature)
    - Перемещает val_frac% файлов каждого класса в val_dir
    - Пишет train_csv (80%) и val_csv (20%)
    Возвращает DataFrame с тренировочной частью.
    """
    df = pd.read_csv(csv_path)
    os.makedirs(val_dir, exist_ok=True)

    train_parts = []
    val_parts = []

    for cls, grp in df.groupby('target_feature'):
        n_val = int(len(grp) * val_frac)
        val_samples = grp.sample(n=n_val, random_state=random_state)
        train_samples = grp.drop(val_samples.index)

        # перемещаем файлы в папку валидации
        for img_id in val_samples['img_id']:
            for ext in ('.jpg', '.jpeg', '.png'):
                src = os.path.join(img_dir, f'{img_id}{ext}')
                if os.path.isfile(src):
                    dst = os.path.join(val_dir, f'{img_id}{ext}')
                    shutil.move(src, dst)
                    break
            else:
                raise FileNotFoundError(f"Не найден файл для {img_id} в {img_dir}")

        train_parts.append(train_samples)
        val_parts.append(val_samples)

    train_df = pd.concat(train_parts, ignore_index=True)
    val_df = pd.concat(val_parts, ignore_index=True)

    train_df.to_csv(train_csv, index=False)
    val_df.to_csv(val_csv, index=False)


    print(f"Split done: {len(val_df)} files moved to {val_dir}.")
    print(f"Train CSV: {train_csv}")
    print(f"Val   CSV: {val_csv}")
    return train_df

def augment_dataset(
    img_dir: str,
    df: pd.DataFrame,
    out_csv: str,
    random_state: int = 42,
    image_size: int = 256
):
    """
    Балансировка классов через аугментацию:
    - df — DataFrame с колонками img_id,target_feature (только train)
    - создаёт в img_dir новые файлы до max_count на класс
    - сохраняет итоговый CSV (старые + новые записи) в out_csv
    """
    counts = df[''].value_counts()
    max_count = counts.max()

    augment = get_offline_augmentations(image_size)
    rng = np.random.RandomState(random_state)
    new_rows = []

    for cls, grp in df.groupby('target_feature'):
        n_exist = len(grp)
        n_need = max_count - n_exist
        if n_need <= 0:
            continue

        sampled = grp.sample(n=n_need, replace=True, random_state=rng)
        for _, row in sampled.iterrows():
            img_id = row['icid_id']
            img = None
            for ext in ('.jpg', '.jpeg', '.png'):
                path = os.path.join(img_dir, f'{img_id}{ext}')
                if os.path.isfile(path):
                    img = cv2.imread(path)
                    break
            if img is None:
                raise FileNotFoundError(f"Не нашёл {img_id} в {img_dir}")

            aug_img = augment(image=img)['image']

            new_id = f"{img_id}_aug_{rng.randint(1_000_000)}"
            dst_path = os.path.join(img_dir, f"{new_id}.jpg")
            cv2.imwrite(dst_path, aug_img)

            new_rows.append({'icid_id': new_id, 'target_feature': cls})

    df_aug = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
    df_aug.to_csv(out_csv, index=False)
    print(f"Balanced to {max_count} per class. Augmented CSV: {out_csv}")

def delete_aug_files(directory: str):
    """
    Опционально: удаляет из directory все файлы, в имени которых 'aug' (case‑insensitive).
    """
    if not os.path.isdir(directory):
        print(f"Ошибка: {directory} не является директорией.")
        return
    for fn in os.listdir(directory):
        if 'aug' in fn.lower():
            path = os.path.join(directory, fn)
            if os.path.isfile(path):
                try:
                    os.remove(path)
                except Exception as e:
                    print(f"Не удалось удалить {path}: {e}")



In [ ]:
IMG_DIR       = 'dataset/img_train'
CSV_PATH      = 'dataset/train_answers.csv'
VAL_DIR       = 'dataset/img_val'
TRAIN_CSV     = 'dataset/train_split.csv'
VAL_CSV       = 'dataset/val_split.csv'
TRAIN_AUG_CSV = 'dataset/train_augmented.csv'


train_df = split_dataset(
    img_dir=IMG_DIR,
    csv_path=CSV_PATH,
    val_dir=VAL_DIR,
    train_csv=TRAIN_CSV,
    val_csv=VAL_CSV,
    val_frac=0.2,
    random_state=42
)

In [ ]:
augment_dataset(
    img_dir=IMG_DIR,
    df=train_df,
    out_csv=TRAIN_AUG_CSV,
    random_state=42,
    image_size=256
)

In [5]:
class MoleDataset(Dataset):
    def __init__(self, csv_file: str, img_dir: str, 
                 img_size: int = 256,
                 transform: transforms.Compose = None):
        self.annotations = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.img_size = img_size

        self.base_transform = transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std =[0.229, 0.224, 0.225]),
        ])

        if transform:
            self.transform = transforms.Compose([
                *self.base_transform.transforms,
                *transform.transforms
            ])
        else:
            self.transform = self.base_transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        row = self.annotations.iloc[idx]
        img_id = row['isic_id']
        target = row['benign_malignant']

        for ext in ('jpg', 'png', 'jpeg'):
            img_path = os.path.join(self.img_dir, f"{img_id}.{ext}")
            if os.path.exists(img_path):
                break
        else:
            raise FileNotFoundError(f"Не найден файл для {img_id} в {self.img_dir}")

        image = Image.open(img_path).convert('RGB')
        image = self.transform(image)

        target_tensor = torch.tensor(target, dtype=torch.long)

        return image, target_tensor


In [6]:
BATCH_SIZE = 64

img_size = 256

train_ds = MoleDataset(
    csv_file='dataset/train_split.csv',
    img_dir='dataset/img_train', 
    img_size=img_size
)

val_ds = MoleDataset(
    csv_file='dataset/val_split.csv', 
    img_dir='dataset/img_val', 
    img_size=img_size
)

train_loader = DataLoader(
    train_ds, 
    batch_size=BATCH_SIZE,
    shuffle=True
)
val_loader = DataLoader(
    val_ds,
    batch_size=BATCH_SIZE,
    shuffle=False
)

In [7]:
class SkinCancerCNN(nn.Sequential):
    def __init__(self):
        super().__init__(
            # Conv Block 1 (256->128)
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 256x256 -> 128x128
            
            # Conv Block 2 (128->64)
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 128x128 -> 64x64
            
            # Conv Block 3 (64->32)
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 64x64 -> 32x32
            
            # Conv Block 4 (32->16) - новый блок!
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 32x32 -> 16x16
            
            # Head
            nn.AdaptiveAvgPool2d(1),  # Глобальный пулинг
            nn.Flatten(),
            nn.Linear(256, 128),  # Увеличили размерность
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 1)
        )
model = SkinCancerCNN().to(device)

In [8]:
def show_metrics(epoch: int, **samples) -> None:
    clear_output(wait=True)

    plt.figure(figsize=(18, 10))
    charts = len(samples)

    for i, sample in enumerate(samples):
        plt.subplot(ceil(charts / 3), 3, i + 1)
        plt.title(sample)
        plt.yscale('log' if sample == 'Loss' else 'linear')
        for metric_name, metric_vals in samples[sample].items():
            plt.plot(range(len(metric_vals)), metric_vals, label=metric_name)
        plt.legend()
    plt.show()

    print(f"\nEpoch {epoch} summary:")
    for name, grp in samples.items():
        tr_last = grp['train'][-1]
        val_last = grp['val'][-1]
        print(f"{name:<9} | train: {tr_last:>.4f} | val: {val_last:>.4f}")

In [ ]:
class TrainingPipeline:
    def __init__(self, model: nn.Module, train_loader: DataLoader,
                 val_loader: DataLoader, criterion: nn.Module,
                 optimizer: optim.Optimizer, device: torch.device,
                 scheduler = None,
                 metrics_visualizer = show_metrics,
                 scheduler_step_per_epoch: bool = True,
                 checkpoint_dir = None,
                 metric_average: str = 'macro'):
        self.model = model.to(device)
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.criterion = criterion
        self.optimizer = optimizer
        self.device = device
        self.scheduler = scheduler
        self.metrics_visualizer = metrics_visualizer
        self.scheduler_step_per_epoch = scheduler_step_per_epoch
        self.checkpoint_dir = checkpoint_dir
        self.metric_average = metric_average

        # Создаем директорию для чекпоинтов, если она указана и не существует
        if self.checkpoint_dir:
            os.makedirs(self.checkpoint_dir, exist_ok=True)

        # Хранилище для метрик
        self.metrics = {
            'Loss': {'train': [], 'val': []},
            'Accuracy': {'train': [], 'val': []},
            'Precision': {'train': [], 'val': []},
            'Recall': {'train': [], 'val': []},
            'F1': {'train': [], 'val': []}
        }

    def _calculate_metrics(self, all_outputs: torch.Tensor, all_labels: torch.Tensor):
        if all_labels.numel() == 0:
            return {'accuracy': 0.0, 'precision': 0.0, 'recall': 0.0, 'f1': 0.0}
        
        # Для бинарной классификации
        predicted = (torch.sigmoid(all_outputs) > 0.5).long()
        
        # Если all_outputs имеет размер [batch_size, 1], нужно squeeze
        if len(all_outputs.shape) > 1 and all_outputs.shape[1] == 1:
            predicted = predicted.squeeze(1)
        
        labels_np = all_labels.cpu().numpy()
        predicted_np = predicted.cpu().numpy()
        
        accuracy = (predicted == all_labels).float().mean().item()
        precision = precision_score(labels_np, predicted_np, average='binary', zero_division=0)
        recall = recall_score(labels_np, predicted_np, average='binary', zero_division=0)
        f1 = f1_score(labels_np, predicted_np, average='binary', zero_division=0)
        
        return {'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1': f1}


    def _run_epoch(self, phase: str):
        is_train = phase == 'train'
        if is_train:
            self.model.train()
            loader = self.train_loader
        else:
            self.model.eval()
            loader = self.val_loader

        running_loss = 0.0
        all_labels_list = []
        all_outputs_list = []
        processed_samples = 0
        counter = 0

        context = torch.enable_grad() if is_train else torch.no_grad()
        with context:
            for inputs, labels in tqdm(loader):
                counter += 1
                inputs, labels = inputs.to(self.device), labels.to(self.device)

                if is_train:
                    self.optimizer.zero_grad()

                outputs = self.model(inputs).squeeze(1) 
                loss = self.criterion(outputs, labels.float()) 

                if is_train:
                    loss.backward()
                    self.optimizer.step()

                batch_size = inputs.size(0)
                running_loss += loss.item() * batch_size
                processed_samples += batch_size

                all_outputs_list.append(outputs.detach().cpu())
                all_labels_list.append(labels.detach().cpu())

        all_outputs_tensor = torch.cat(all_outputs_list, dim=0) if all_outputs_list else torch.empty(0)
        all_labels_tensor = torch.cat(all_labels_list, dim=0) if all_labels_list else torch.empty(0)

        epoch_loss = running_loss / processed_samples if processed_samples > 0 else 0.0
        epoch_metrics = self._calculate_metrics(all_outputs_tensor, all_labels_tensor)

        results = {'loss': epoch_loss, **epoch_metrics}
        return results

    def _save_checkpoint(self, epoch: int, val_loss: float):
        if not self.checkpoint_dir:
            return

        checkpoint_path = os.path.join(self.checkpoint_dir, f'model_epoch_{epoch}.pth')
        state = {
            'epoch': epoch,
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'val_loss': val_loss,
        }
        if self.scheduler:
            state['scheduler_state_dict'] = self.scheduler.state_dict()

        torch.save(state, checkpoint_path)

    def run_training(self, num_epochs: int):
        for epoch in range(1, num_epochs + 1):
            train_res = self._run_epoch('train')
            val_res   = self._run_epoch('val')

            for k, alias in (('loss', 'Loss'),
                             ('accuracy', 'Accuracy'),
                             ('precision', 'Precision'),
                             ('recall', 'Recall'),
                             ('f1',  'F1')):
                self.metrics[alias]['train'].append(train_res[k])
                self.metrics[alias]['val'].append(val_res[k])

            if self.scheduler:
                if self.scheduler_step_per_epoch and not isinstance(
                        self.scheduler, optim.lr_scheduler.ReduceLROnPlateau):
                    self.scheduler.step()
                elif isinstance(self.scheduler, optim.lr_scheduler.ReduceLROnPlateau):
                    self.scheduler.step(val_res['loss'])

            if self.checkpoint_dir:
                self._save_checkpoint(epoch, val_res['loss'])

            if self.metrics_visualizer:
                self.metrics_visualizer(epoch, **self.metrics)

        print("Training finished.")
        return self.metrics


In [ ]:
checkpoint = torch.load('tested_models/checkpoints_ConvNeXt9.1/model_epoch_17.pth')['model_state_dict']
model.load_state_dict(checkpoint)

In [10]:
model

SkinCancerCNN(
  (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): ReLU()
  (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (10): ReLU()
  (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (12): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (14): ReLU()
  (15): MaxPool2d(kernel_size=2, s

In [15]:
pos_weight = torch.tensor([5647/4003]).to(device)  # ~1.41
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3)
num_epochs = 30

In [16]:
pipeline = TrainingPipeline(model=model,
                            train_loader=train_loader,
                            val_loader=val_loader,
                            criterion=criterion,
                            optimizer=optimizer,
                            device=device,
                            scheduler=scheduler,
                            metrics_visualizer=show_metrics,
                            scheduler_step_per_epoch=True,
                            checkpoint_dir='tested_models/checkpoints_CNN1')


In [17]:
final_metrics = pipeline.run_training(num_epochs=num_epochs)

print("\nFinal Metrics History:")
print(final_metrics)

100%|██████████| 121/121 [01:34<00:00,  1.29it/s]


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)